# Dados cartográficos com Python

## GeoPandas

GeoPandas é uma biblioteca do Python que estende as funcionalidades do Pandas, permitindo que se trabalhe de maneira eficiente com dados geoespaciais. Ela é construída sobre outras bibliotecas populares como Shapely, Fiona e Pyproj, o que lhe permite manipular, analisar e visualizar dados geoespaciais de forma integrada e simplificada.

A principal característica do GeoPandas é a capacidade de lidar com geometrias (pontos, linhas, polígonos, etc.) em colunas especiais chamadas de `GeoSeries`. Essas geometrias podem ser usadas para representar entidades espaciais, como localizações geográficas, fronteiras de cidades, ou trajetórias. Cada linha de um `GeoDataFrame`—a versão geoespacial do `DataFrame` no Pandas—pode conter uma geometria associada a outros atributos, permitindo que se façam análises espaciais sobre os dados.

Com o GeoPandas, tarefas comuns em análise geoespacial, como a leitura de arquivos shapefile, a re-projeção de sistemas de coordenadas e operações espaciais (interseção, união, diferenciação) tornam-se muito mais fáceis. A biblioteca é amplamente usada em aplicações que envolvem geoprocessamento, cartografia e estudos de geografia.

GeoPandas também se integra bem com bibliotecas de visualização como Matplotlib, facilitando a criação de mapas e gráficos baseados em dados espaciais. Ela é amplamente usada em estudos ambientais, urbanos, logísticos e em qualquer área que necessite de análise espacial eficiente.

https://geopandas.org/en/stable/index.html

## Folium

Folium é uma biblioteca do Python voltada para a criação de mapas interativos utilizando Leaflet.js, uma popular biblioteca JavaScript de mapeamento. O Folium permite que se criem mapas ricos e dinâmicos diretamente em Python, integrando dados geoespaciais de maneira simples e intuitiva.

Uma das principais vantagens do Folium é a facilidade com que ele transforma dados geoespaciais, como pontos de interesse, trajetos ou áreas geográficas, em visualizações interativas. Ele suporta uma ampla gama de camadas, como marcadores, polígonos, retângulos e círculos, que podem ser facilmente sobrepostos em diferentes tipos de mapas base, como mapas de ruas ou satélites. Além disso, a biblioteca permite adicionar informações complementares por meio de popups e tooltips, o que facilita a apresentação de dados complexos de forma clara e acessível.

Folium é frequentemente utilizado em projetos que requerem visualizações de dados espaciais interativas, como análise de trajetórias, planejamento urbano, monitoramento ambiental e estudos logísticos. Sua integração com outras bibliotecas como Pandas e GeoPandas permite manipular dados e criar mapas de forma eficiente. A biblioteca também facilita a exportação dos mapas para HTML, o que é útil para incorporá-los em sites ou relatórios interativos.

De maneira geral, Folium oferece uma solução para transformar dados geoespaciais em mapas interativos, sendo uma escolha popular tanto em análises exploratórias quanto na apresentação final de resultados.

https://python-visualization.github.io/folium/latest/#

In [1]:
import folium
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from folium import plugins
from shapely.geometry import Point

from src.config import DADOS_LIMPOS, DADOS_GEO_ORIGINAIS
from src.graficos import SCATTER_ALPHA

sns.set_theme(style="white", palette="bright")

In [2]:
df = pd.read_parquet(DADOS_LIMPOS)

df.head()

longitude   latitude  housing_median_age  total_rooms  total_bedrooms  \
0 -122.230003  37.880001                  41          880             129   
1 -122.220001  37.860001                  21         7099            1106   
2 -122.260002  37.840000                  42         2555             665   
3 -122.260002  37.849998                  50         1120             283   
4 -122.260002  37.840000                  50         2239             455   

   population  households  median_income  median_house_value ocean_proximity  \
0         322         126         8.3252              452600        NEAR BAY   
1        2401        1138         8.3014              358500        NEAR BAY   
2        1206         595         2.0804              226700        NEAR BAY   
3         697         264         2.1250              140000        NEAR BAY   
4         990         419         1.9911              158700        NEAR BAY   

   median_income_cat  rooms_per_household  population_per_household  \
0                  5             6.984127                  2.555556   
1                  5             6.238137                  2.109842   
2                  2             4.294117                  2.026891   
3                  2             4.242424                  2.640152   
4                  2             5.343676                  2.362768   

   bedrooms_per_room  
0           0.146591  
1           0.155797  
2           0.260274  
3           0.252679  
4           0.203216

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17564 entries, 0 to 17563
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   longitude                 17564 non-null  float32 
 1   latitude                  17564 non-null  float32 
 2   housing_median_age        17564 non-null  int8    
 3   total_rooms               17564 non-null  int16   
 4   total_bedrooms            17564 non-null  int16   
 5   population                17564 non-null  int16   
 6   households                17564 non-null  int16   
 7   median_income             17564 non-null  float32 
 8   median_house_value        17564 non-null  int32   
 9   ocean_proximity           17564 non-null  category
 10  median_income_cat         17564 non-null  int64   
 11  rooms_per_household       17564 non-null  float32 
 12  population_per_household  17564 non-null  float32 
 13  bedrooms_per_room         17564 non-null  floa

O melhor termo para traduzir **"county"** seria **"condado"**. Embora o Brasil não tenha uma divisão administrativa exata equivalente aos condados dos Estados Unidos, "condado" é uma tradução apropriada e amplamente compreendida.

Nos EUA, os **counties** (condados) são subdivisões dos estados, com certa autonomia administrativa, similar às regiões intermediárias entre estados e municípios no Brasil. No entanto, a melhor correspondência prática no Brasil, considerando a divisão político-administrativa, seria com os **municípios**, que são as subdivisões administrativas dos estados brasileiros, apesar de suas diferenças em termos de autonomia e funções.


In [5]:
gdf_counties = gpd.read_file(DADOS_GEO_ORIGINAIS)

gdf_counties.head()

name          fullname abbrev abcode ansi  \
0   Siskiyou   Siskiyou County    SIS   c047  093   
1  Del Norte  Del Norte County    DNT   c008  015   
2      Modoc      Modoc County    MOD   c025  049   
3    Trinity    Trinity County    TRI   c053  105   
4     Shasta     Shasta County    SHA   c045  089   

                                            geometry  
0  POLYGON ((-121.44597 41.1839, -121.46238 41.18...  
1  MULTIPOLYGON (((-124.19097 41.73619, -124.1933...  
2  POLYGON ((-121.44784 41.99739, -121.42322 41.9...  
3  POLYGON ((-122.66795 40.77637, -122.66814 40.7...  
4  POLYGON ((-122.17328 40.37906, -122.17359 40.3...

Em um arquivo GeoJSON, colunas como **"abcode"** e **"ansi"** contêm códigos padronizados que identificam de maneira única regiões geográficas, divisões administrativas ou entidades espaciais.

A coluna **"abcode"** representa **códigos de fronteira administrativa** (Administrative Boundary Code). Esses códigos são usados para identificar divisões administrativas, como estados, condados, cidades ou outras subdivisões de um país. Dependendo do contexto, "abcode" pode estar relacionado a códigos nacionais ou internacionais de áreas administrativas.

Por exemplo:
- Nos Estados Unidos, o "abcode" pode se referir a códigos FIPS (Federal Information Processing Standards), usados para identificar estados e condados.
- Em outros países, pode ser um código específico para uma província, estado ou outra divisão territorial.

A coluna **"ansi"** refere-se aos códigos do **American National Standards Institute (ANSI)** (American National Standards Institute Code), usados nos Estados Unidos para identificar regiões geográficas. O ANSI substituiu os códigos FIPS em 2008 como padrão de identificação de entidades geográficas, como estados e condados.

Por exemplo:
- Nos EUA, cada estado e condado tem um **ANSI code** único que os identifica de forma oficial. Para estados, o ANSI code é o mesmo que o código FIPS (geralmente um número de 2 dígitos), e para condados, é um número de 5 dígitos.

Assim: 

- **abcode**: Refere-se a um código de fronteira administrativa, usado para identificar divisões geográficas ou administrativas específicas, como estados, cidades ou condados.
- **ansi**: Refere-se a códigos padronizados pelo ANSI, usados principalmente nos EUA para identificar estados e condados. Esses códigos são uma forma de identificar regiões geográficas de maneira única e padronizada.

### Sobre sistemas de referência

Um Sistema de Referência de Coordenadas (CRS, do inglês *Coordinate Reference System*) é o conjunto de regras que define como os dados geoespaciais são projetados na superfície da Terra. Imagine o CRS como uma forma de "tradução" que nos ajuda a entender e posicionar os dados geográficos corretamente em um mapa. Cada ponto no mapa (como uma cidade, uma montanha ou uma estrada) tem coordenadas que dependem do CRS usado. Essas coordenadas podem ser expressas em diferentes sistemas, como latitudes e longitudes ou distâncias em metros.

Por que isso é importante? Quando trabalhamos com dados geoespaciais, especialmente em formatos como GeoJSON, Shapefiles ou outros, o CRS nos diz como interpretar as coordenadas. Por exemplo, o sistema mais comum que você verá é o WGS 84 (EPSG:4326), que usa latitude e longitude para posicionar qualquer ponto na Terra. No entanto, em projetos locais ou regionais, outros sistemas de referência podem ser usados, que distorcem menos as áreas específicas.

Quando você abre um arquivo GeoJSON ou outro tipo de dado geoespacial, muitas vezes verá algo como "crs": {"type": "name", "properties": {"name": "EPSG:4326"}}. Isso está simplesmente informando qual sistema está sendo usado para mapear as coordenadas no arquivo. Se diferentes arquivos estiverem em CRSs distintos, será necessário convertê-los para o mesmo sistema, ou eles não se alinharão corretamente quando visualizados juntos.



**WGS 84** é um sistema geodésico global que define um modelo matemático para a forma da Terra. Ele inclui um elipsoide de referência (uma representação matemática da Terra), um sistema de coordenadas cartesianas (X, Y, Z) e um sistema de coordenadas geográficas (latitude, longitude e altura). Esse sistema é amplamente usado em GPS e outros sistemas de posicionamento global, sendo uma das referências mais comuns para medir e mapear a superfície da Terra.

**EPSG:4326**, por outro lado, é o código de identificação que faz parte de uma base de dados chamada **EPSG Registry**, que contém descrições de vários sistemas de referência de coordenadas usados globalmente. O código **4326** se refere especificamente ao CRS que utiliza o **WGS 84** como sistema geodésico de base e expressa coordenadas em graus de latitude e longitude. Esse código é amplamente utilizado em formatos geoespaciais, como GeoJSON, para identificar que o sistema de coordenadas é o WGS 84.

Então, podemos dizer que **WGS 84** é o sistema geodésico, e **EPSG:4326** é o código que referencia esse sistema específico dentro de um conjunto de padrões geoespaciais. Eles estão intimamente relacionados, pois EPSG:4326 usa WGS 84, mas o EPSG:4326 é apenas uma forma de identificar esse sistema em softwares e arquivos.

#### Relação com as projeções de mapas

A Terra tem uma forma quase esférica (na verdade, ela é um esferoide oblato, ligeiramente achatada nos polos), mas os mapas são geralmente representações bidimensionais. Isso gera um desafio: como transformar a superfície curva da Terra em um mapa plano? É aí que entram as **projeções cartográficas**.

**Projeção cartográfica** é o método utilizado para "achatar" a superfície da Terra em um plano, permitindo a criação de mapas. Cada projeção tenta preservar algumas características da Terra (como áreas, formas ou distâncias), mas sempre há algum tipo de distorção, já que é impossível representar uma superfície curva em um plano sem perdas.

Agora, voltando ao **WGS 84** e **EPSG:4326**:

- **WGS 84** é um sistema de referência global que define coordenadas em latitude e longitude, como mencionado antes, sem aplicar diretamente uma projeção cartográfica. Ou seja, ele usa um sistema geodésico para definir onde os pontos estão na superfície curva da Terra, sem "achatar" essa superfície em um mapa.

- Quando usamos **EPSG:4326**, estamos trabalhando com coordenadas de latitude e longitude, mas sem projetar essas coordenadas em uma forma plana (ainda estamos no sistema esférico). Essa é uma das formas mais comuns de representar dados geoespaciais porque é universal e fácil de entender, mas não é exatamente uma projeção cartográfica.

As projeções cartográficas que aprendemos na escola—como a **projeção de Mercator** (que distorce áreas perto dos polos), a **projeção de Peters** (que tenta preservar áreas), ou a **projeção azimutal** (que preserva direções a partir de um ponto central)—são formas de transformar as coordenadas da Terra em um plano. Dependendo da finalidade do mapa, uma projeção pode ser mais adequada que outra.

Por exemplo:

- A **projeção de Mercator** é útil para navegação porque preserva ângulos e direções, mas distorce o tamanho das regiões conforme se aproximam dos polos.
- A **projeção de Peters** tenta representar as áreas de forma mais precisa, mas distorce as formas dos continentes.

Quando usamos um CRS como o **EPSG:4326** (WGS 84), estamos basicamente posicionando pontos na superfície curva da Terra. Para "achatar" esses pontos em um mapa, muitas vezes usamos uma projeção cartográfica. Por exemplo, se quisermos criar um mapa que use uma projeção de Mercator, utilizamos outro CRS específico para essa projeção, como o **EPSG:3857**, que é a projeção de Mercator.

Em resumo, os conceitos de sistemas de referência de coordenadas (CRS) e projeções cartográficas estão conectados. O CRS nos dá as coordenadas na superfície curva da Terra, e a projeção cartográfica define como essas coordenadas serão representadas em um mapa plano.

In [6]:
df.head()

longitude   latitude  housing_median_age  total_rooms  total_bedrooms  \
0 -122.230003  37.880001                  41          880             129   
1 -122.220001  37.860001                  21         7099            1106   
2 -122.260002  37.840000                  42         2555             665   
3 -122.260002  37.849998                  50         1120             283   
4 -122.260002  37.840000                  50         2239             455   

   population  households  median_income  median_house_value ocean_proximity  \
0         322         126         8.3252              452600        NEAR BAY   
1        2401        1138         8.3014              358500        NEAR BAY   
2        1206         595         2.0804              226700        NEAR BAY   
3         697         264         2.1250              140000        NEAR BAY   
4         990         419         1.9911              158700        NEAR BAY   

   median_income_cat  rooms_per_household  population_per_household  \
0                  5             6.984127                  2.555556   
1                  5             6.238137                  2.109842   
2                  2             4.294117                  2.026891   
3                  2             4.242424                  2.640152   
4                  2             5.343676                  2.362768   

   bedrooms_per_room  
0           0.146591  
1           0.155797  
2           0.260274  
3           0.252679  
4           0.203216

In [7]:
pontos = [Point(long, lat) for long, lat in zip(df["longitude"], df["latitude"])]

pontos[:5]

[<POINT (-122.23 37.88)>,
 <POINT (-122.22 37.86)>,
 <POINT (-122.26 37.84)>,
 <POINT (-122.26 37.85)>,
 <POINT (-122.26 37.84)>]

In [8]:
gdf = gpd.GeoDataFrame(df, geometry=pontos)

gdf.head()

longitude   latitude  housing_median_age  total_rooms  total_bedrooms  \
0 -122.230003  37.880001                  41          880             129   
1 -122.220001  37.860001                  21         7099            1106   
2 -122.260002  37.840000                  42         2555             665   
3 -122.260002  37.849998                  50         1120             283   
4 -122.260002  37.840000                  50         2239             455   

   population  households  median_income  median_house_value ocean_proximity  \
0         322         126         8.3252              452600        NEAR BAY   
1        2401        1138         8.3014              358500        NEAR BAY   
2        1206         595         2.0804              226700        NEAR BAY   
3         697         264         2.1250              140000        NEAR BAY   
4         990         419         1.9911              158700        NEAR BAY   

   median_income_cat  rooms_per_household  population_per_household  \
0                  5             6.984127                  2.555556   
1                  5             6.238137                  2.109842   
2                  2             4.294117                  2.026891   
3                  2             4.242424                  2.640152   
4                  2             5.343676                  2.362768   

   bedrooms_per_room               geometry  
0           0.146591  POINT (-122.23 37.88)  
1           0.155797  POINT (-122.22 37.86)  
2           0.260274  POINT (-122.26 37.84)  
3           0.252679  POINT (-122.26 37.85)  
4           0.203216  POINT (-122.26 37.84)

In [9]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 17564 entries, 0 to 17563
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   longitude                 17564 non-null  float32 
 1   latitude                  17564 non-null  float32 
 2   housing_median_age        17564 non-null  int8    
 3   total_rooms               17564 non-null  int16   
 4   total_bedrooms            17564 non-null  int16   
 5   population                17564 non-null  int16   
 6   households                17564 non-null  int16   
 7   median_income             17564 non-null  float32 
 8   median_house_value        17564 non-null  int32   
 9   ocean_proximity           17564 non-null  category
 10  median_income_cat         17564 non-null  int64   
 11  rooms_per_household       17564 non-null  float32 
 12  population_per_household  17564 non-null  float32 
 13  bedrooms_per_room         17564 non-nu

In [11]:
gdf.crs

In [10]:
gdf_counties.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
gdf = gdf.set_crs(epsg=4326)

gdf_counties = gdf_counties.to_crs(epsg=4326)

In [13]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
gdf_counties.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins

https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html

In [15]:
gdf_joined = gpd.sjoin(gdf, gdf_counties, how="left", predicate="within")

gdf_joined.head()

longitude   latitude  housing_median_age  total_rooms  total_bedrooms  \
0 -122.230003  37.880001                  41          880             129   
1 -122.220001  37.860001                  21         7099            1106   
2 -122.260002  37.840000                  42         2555             665   
3 -122.260002  37.849998                  50         1120             283   
4 -122.260002  37.840000                  50         2239             455   

   population  households  median_income  median_house_value ocean_proximity  \
0         322         126         8.3252              452600        NEAR BAY   
1        2401        1138         8.3014              358500        NEAR BAY   
2        1206         595         2.0804              226700        NEAR BAY   
3         697         264         2.1250              140000        NEAR BAY   
4         990         419         1.9911              158700        NEAR BAY   

   ...  rooms_per_household  population_per_household  bedrooms_per_room  \
0  ...             6.984127                  2.555556           0.146591   
1  ...             6.238137                  2.109842           0.155797   
2  ...             4.294117                  2.026891           0.260274   
3  ...             4.242424                  2.640152           0.252679   
4  ...             5.343676                  2.362768           0.203216   

                geometry index_right     name        fullname abbrev abcode  \
0  POINT (-122.23 37.88)        34.0  Alameda  Alameda County    ALA   c001   
1  POINT (-122.22 37.86)        34.0  Alameda  Alameda County    ALA   c001   
2  POINT (-122.26 37.84)        34.0  Alameda  Alameda County    ALA   c001   
3  POINT (-122.26 37.85)        34.0  Alameda  Alameda County    ALA   c001   
4  POINT (-122.26 37.84)        34.0  Alameda  Alameda County    ALA   c001   

  ansi  
0  001  
1  001  
2  001  
3  001  
4  001  

[5 rows x 21 columns]

In [16]:
gdf_joined.tail()

longitude   latitude  housing_median_age  total_rooms  total_bedrooms  \
17559 -121.089996  39.480000                  25         1665             374   
17560 -121.209999  39.490002                  18          697             150   
17561 -121.220001  39.430000                  17         2254             485   
17562 -121.320000  39.430000                  18         1860             409   
17563 -121.239998  39.369999                  16         2785             616   

       population  households  median_income  median_house_value  \
17559         845         330         1.5603               78100   
17560         356         114         2.5568               77100   
17561        1007         433         1.7000               92300   
17562         741         349         1.8672               84700   
17563        1387         530         2.3886               89400   

      ocean_proximity  ...  rooms_per_household  population_per_household  \
17559          INLAND  ...             5.045455                  2.560606   
17560          INLAND  ...             6.114035                  3.122807   
17561          INLAND  ...             5.205543                  2.325635   
17562          INLAND  ...             5.329513                  2.123209   
17563          INLAND  ...             5.254717                  2.616981   

       bedrooms_per_room               geometry index_right  name  \
17559           0.224625  POINT (-121.09 39.48)        12.0  Yuba   
17560           0.215208  POINT (-121.21 39.49)        12.0  Yuba   
17561           0.215173  POINT (-121.22 39.43)        12.0  Yuba   
17562           0.219892  POINT (-121.32 39.43)        12.0  Yuba   
17563           0.221185  POINT (-121.24 39.37)        12.0  Yuba   

          fullname abbrev abcode ansi  
17559  Yuba County    YUB   c058  115  
17560  Yuba County    YUB   c058  115  
17561  Yuba County    YUB   c058  115  
17562  Yuba County    YUB   c058  115  
17563  Yuba County    YUB   c058  115  

[5 rows x 21 columns]

In [17]:
gdf_counties.iloc[12]

name                                                     Yuba
fullname                                          Yuba County
abbrev                                                    YUB
abcode                                                   c058
ansi                                                      115
geometry    POLYGON ((-121.01794117107198 39.5919167593742...
Name: 12, dtype: object

In [18]:
gdf_joined = gdf_joined.drop(
    columns=["index_right", "fullname", "abcode", "ansi"]
)

gdf_joined.head()

longitude   latitude  housing_median_age  total_rooms  total_bedrooms  \
0 -122.230003  37.880001                  41          880             129   
1 -122.220001  37.860001                  21         7099            1106   
2 -122.260002  37.840000                  42         2555             665   
3 -122.260002  37.849998                  50         1120             283   
4 -122.260002  37.840000                  50         2239             455   

   population  households  median_income  median_house_value ocean_proximity  \
0         322         126         8.3252              452600        NEAR BAY   
1        2401        1138         8.3014              358500        NEAR BAY   
2        1206         595         2.0804              226700        NEAR BAY   
3         697         264         2.1250              140000        NEAR BAY   
4         990         419         1.9911              158700        NEAR BAY   

   median_income_cat  rooms_per_household  population_per_household  \
0                  5             6.984127                  2.555556   
1                  5             6.238137                  2.109842   
2                  2             4.294117                  2.026891   
3                  2             4.242424                  2.640152   
4                  2             5.343676                  2.362768   

   bedrooms_per_room               geometry     name abbrev  
0           0.146591  POINT (-122.23 37.88)  Alameda    ALA  
1           0.155797  POINT (-122.22 37.86)  Alameda    ALA  
2           0.260274  POINT (-122.26 37.84)  Alameda    ALA  
3           0.252679  POINT (-122.26 37.85)  Alameda    ALA  
4           0.203216  POINT (-122.26 37.84)  Alameda    ALA

In [19]:
gdf_joined.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 17564 entries, 0 to 17563
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   longitude                 17564 non-null  float32 
 1   latitude                  17564 non-null  float32 
 2   housing_median_age        17564 non-null  int8    
 3   total_rooms               17564 non-null  int16   
 4   total_bedrooms            17564 non-null  int16   
 5   population                17564 non-null  int16   
 6   households                17564 non-null  int16   
 7   median_income             17564 non-null  float32 
 8   median_house_value        17564 non-null  int32   
 9   ocean_proximity           17564 non-null  category
 10  median_income_cat         17564 non-null  int64   
 11  rooms_per_household       17564 non-null  float32 
 12  population_per_household  17564 non-null  float32 
 13  bedrooms_per_room         17564 non-null  f

In [20]:
gdf_joined.isnull().sum()

longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms                0
population                    0
households                    0
median_income                 0
median_house_value            0
ocean_proximity               0
median_income_cat             0
rooms_per_household           0
population_per_household      0
bedrooms_per_room             0
geometry                      0
name                        148
abbrev                      148
dtype: int64

In [21]:
gdf_joined[gdf_joined.isnull().any(axis=1)]

longitude   latitude  housing_median_age  total_rooms  total_bedrooms  \
1507  -124.300003  41.799999                  19         2672             552   
1508  -124.230003  41.750000                  11         3159             616   
1511  -124.220001  41.730000                  28         3003             699   
1517  -124.300003  41.840000                  17         2677             531   
1521  -119.940002  38.959999                  20         1451             386   
...           ...        ...                 ...          ...             ...   
17198 -119.300003  34.270000                  17         1527             503   
17199 -119.290001  34.259998                  32         3295             764   
17201 -119.290001  34.230000                  22         2486             608   
17232 -119.230003  34.150002                  18         6213            1188   
17258 -119.209999  34.119999                  15         5778            1285   

       population  households  median_income  median_house_value  \
1507         1298         478         1.9797               85800   
1508         1343         479         2.4805               73200   
1511         1530         653         1.7038               78300   
1517         1244         456         3.0313              103600   
1521          467         255         1.5536              212500   
...           ...         ...            ...                 ...   
17198         688         423         1.6007              187500   
17199        1344         600         3.6007              395500   
17201         709         523         2.9018              275000   
17232        2679        1000         3.7480              380400   
17258        1722         829         4.3427              305800   

      ocean_proximity  median_income_cat  rooms_per_household  \
1507       NEAR OCEAN                  2             5.589958   
1508       NEAR OCEAN                  2             6.594990   
1511       NEAR OCEAN                  2             4.598775   
1517       NEAR OCEAN                  3             5.870614   
1521           INLAND                  2             5.690196   
...               ...                ...                  ...   
17198      NEAR OCEAN                  2             3.609929   
17199      NEAR OCEAN                  3             5.491667   
17201      NEAR OCEAN                  2             4.753346   
17232      NEAR OCEAN                  3             6.213000   
17258      NEAR OCEAN                  3             6.969843   

       population_per_household  bedrooms_per_room               geometry  \
1507                   2.715481           0.206587    POINT (-124.3 41.8)   
1508                   2.803758           0.194998  POINT (-124.23 41.75)   
1511                   2.343032           0.232767  POINT (-124.22 41.73)   
1517                   2.728070           0.198356   POINT (-124.3 41.84)   
1521                   1.831372           0.266023  POINT (-119.94 38.96)   
...                         ...                ...                    ...   
17198                  1.626478           0.329404   POINT (-119.3 34.27)   
17199                  2.240000           0.231866  POINT (-119.29 34.26)   
17201                  1.355641           0.244570  POINT (-119.29 34.23)   
17232                  2.679000           0.191212  POINT (-119.23 34.15)   
17258                  2.077201           0.222395  POINT (-119.21 34.12)   

      name abbrev  
1507   NaN    NaN  
1508   NaN    NaN  
1511   NaN    NaN  
1517   NaN    NaN  
1521   NaN    NaN  
...    ...    ...  
17198  NaN    NaN  
17199  NaN    NaN  
17201  NaN    NaN  
17232  NaN    NaN  
17258  NaN    NaN  

[148 rows x 17 columns]

In [22]:
linhas_faltantes = gdf_joined[gdf_joined.isnull().any(axis=1)].index

linhas_faltantes

Index([ 1507,  1508,  1511,  1517,  1521,  2154,  2193,  2201,  2203,  2219,
       ...
       15747, 15748, 15752, 15755, 15756, 17198, 17199, 17201, 17232, 17258],
      dtype='int64', length=148)

O **centroide** em dados geográficos é o ponto que representa o centro geométrico de uma forma ou área. Em termos simples, é o ponto médio de uma geometria, como um polígono ou multipolígono, que pode representar uma cidade, um estado ou qualquer outra divisão geográfica.

No contexto do **GeoPandas**, o centroide é uma propriedade útil para resumir a localização de uma área. Por exemplo, se você tem o contorno de um município ou de uma região, o centroide é o ponto que melhor representa o "centro" dessa área, mesmo que sua forma seja irregular. O centroide é calculado considerando as coordenadas da geometria.

No **GeoPandas**, você pode calcular o centroide de uma geometria utilizando o atributo `.centroid`. Isso gera um ponto, que pode ser usado para várias finalidades, como identificar o ponto central de uma área no mapa, fazer comparações de distâncias entre diferentes regiões, ou usá-lo em visualizações.

Por exemplo, se você tem um conjunto de regiões geográficas e deseja identificar seus pontos centrais para colocar marcadores em um mapa, o centroide é ideal para essa finalidade.

- Se você tem o contorno de um país ou estado, o centroide será o ponto mais central dessa área.
- Mesmo para áreas irregulares, como fronteiras naturais ou regiões com formatos complexos, o centroide ainda é o ponto médio calculado geometricamente.


In [23]:
gdf_counties["centroid"] = gdf_counties.centroid

gdf_counties.head()

name          fullname abbrev abcode ansi  \
0   Siskiyou   Siskiyou County    SIS   c047  093   
1  Del Norte  Del Norte County    DNT   c008  015   
2      Modoc      Modoc County    MOD   c025  049   
3    Trinity    Trinity County    TRI   c053  105   
4     Shasta     Shasta County    SHA   c045  089   

                                            geometry  \
0  POLYGON ((-121.44597 41.1839, -121.46238 41.18...   
1  MULTIPOLYGON (((-124.19097 41.73619, -124.1933...   
2  POLYGON ((-121.44784 41.99739, -121.42322 41.9...   
3  POLYGON ((-122.66795 40.77637, -122.66814 40.7...   
4  POLYGON ((-122.17328 40.37906, -122.17359 40.3...   

                      centroid  
0  POINT (-122.54047 41.59268)  
1  POINT (-123.89681 41.74299)  
2  POINT (-120.72517 41.58987)  
3  POINT (-123.11264 40.65063)  
4  POINT (-122.04016 40.76344)

In [28]:
gdf_counties.shape

(58, 7)

In [32]:
gdf_joined.loc[1507]

longitude                                                     -124.300003
latitude                                                        41.799999
housing_median_age                                                     19
total_rooms                                                          2672
total_bedrooms                                                        552
population                                                           1298
households                                                            478
median_income                                                      1.9797
median_house_value                                                  85800
ocean_proximity                                                NEAR OCEAN
median_income_cat                                                       2
rooms_per_household                                              5.589958
population_per_household                                         2.715481
bedrooms_per_room                     

In [26]:
print(gdf_joined.loc[1507, "geometry"])

POINT (-124.30000305175781 41.79999923706055)


In [30]:
gdf_counties["centroid"].distance(gdf_joined.loc[1507, "geometry"])

0      1.771700
1      0.407204
2      3.581004
3      1.652539
4      2.486229
5      3.872982
6      1.181051
7      2.658962
8      3.899362
9      3.440403
10     2.913231
11     4.386820
12     3.885950
13     3.111850
14     4.326303
15     3.336570
16     2.528645
17     3.799275
18     4.508214
19     4.834879
20     3.930031
21     5.506539
22     3.568571
23     3.837241
24     4.467937
25     6.649118
26     4.955842
27     4.248938
28     5.192143
29     5.754299
30     4.910558
31     4.069320
32     4.533433
33     5.374753
34     4.784590
35     6.091522
36     6.448327
37     4.438089
38     4.775127
39     5.836867
40     6.858891
41     5.258028
42     8.685359
43     5.271193
44     6.114297
45     7.834746
46     7.271996
47     6.366951
48    10.694858
49     8.527644
50     7.503716
51     9.007875
52     8.314811
53    11.571357
54    10.407882
55    12.512826
56     9.633791
57    11.578843
dtype: float64

In [29]:
gdf_counties["centroid"].distance(gdf_joined.loc[1507, "geometry"]).idxmin()

1

In [31]:
gdf_counties.loc[1]

name                                                Del Norte
fullname                                     Del Norte County
abbrev                                                    DNT
abcode                                                   c008
ansi                                                      015
geometry    MULTIPOLYGON (((-124.19097394913307 41.7361877...
centroid       POINT (-123.89681032594169 41.742987539381424)
Name: 1, dtype: object

In [35]:
def condado_mais_proximo(linha):
    ponto = linha["geometry"]
    distancias = gdf_counties["centroid"].distance(ponto)
    idx_condado_mais_proximo = distancias.idxmin()
    condado_mais_proximo = gdf_counties.loc[idx_condado_mais_proximo]
    return condado_mais_proximo[["name", "abbrev"]]

In [37]:
condado_mais_proximo(gdf_joined.loc[17232])

name      Ventura
abbrev        VEN
Name: 51, dtype: object

In [38]:
gdf_joined.loc[linhas_faltantes, ["name", "abbrev"]] = gdf_joined.loc[linhas_faltantes].apply(condado_mais_proximo, axis=1)

In [39]:
gdf_joined.isnull().sum()

longitude                   0
latitude                    0
housing_median_age          0
total_rooms                 0
total_bedrooms              0
population                  0
households                  0
median_income               0
median_house_value          0
ocean_proximity             0
median_income_cat           0
rooms_per_household         0
population_per_household    0
bedrooms_per_room           0
geometry                    0
name                        0
abbrev                      0
dtype: int64

In [40]:
gdf_joined.loc[linhas_faltantes, ["name", "abbrev"]]

name abbrev
1507   Del Norte    DNT
1508   Del Norte    DNT
1511   Del Norte    DNT
1517   Del Norte    DNT
1521      Alpine    ALP
...          ...    ...
17198    Ventura    VEN
17199    Ventura    VEN
17201    Ventura    VEN
17232    Ventura    VEN
17258    Ventura    VEN

[148 rows x 2 columns]

In [41]:
gdf_joined.loc[linhas_faltantes, ["name", "abbrev"]].value_counts()

name             abbrev
San Diego        SDG       29
Orange           ORA       27
Santa Cruz       SCR       20
Santa Barbara    SBA       13
San Francisco    SFO       11
San Luis Obispo  SLO       10
Ventura          VEN        9
Los Angeles      LAN        6
Humboldt         HUM        5
Del Norte        DNT        5
Mendocino        MEN        5
Monterey         MNT        4
San Mateo        SMT        2
Marin            MRN        1
Alpine           ALP        1
Name: count, dtype: int64

In [43]:
gdf_joined["name"].value_counts()

name
Los Angeles        4827
San Diego          1467
Orange             1438
Santa Clara         824
Alameda             737
San Bernardino      729
Sacramento          622
Contra Costa        558
Riverside           504
Fresno              491
San Joaquin         387
Kern                353
Ventura             343
San Mateo           336
Sonoma              293
Stanislaus          282
Santa Barbara       261
Tulare              246
San Francisco       213
Santa Cruz          186
Solano              178
Monterey            170
Butte               144
San Luis Obispo     128
Marin               123
Merced              123
Shasta              119
Imperial            115
Humboldt            109
Placer              101
El Dorado           101
Napa                 94
Kings                86
Lake                 81
Nevada               80
Mendocino            77
Yolo                 74
Madera               74
Sutter               56
Yuba                 51
Tehama               47
Tuolumne   